In [5]:
import numpy as np
import pandas as pd

from capture_image import capture_image
import sys
import os
import warnings
import numpy as np
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time
import numpy as np
import rotation_matrix as rm
import pandas as pd

# add parent folder to sys.path
sys.path.insert(0, os.path.abspath(".."))

# ignore DeprecationWarning messages
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ignore RuntimeWarning messages
warnings.filterwarnings("ignore", category=RuntimeWarning)

from volume_estimation.src.models_1_no_vol.predict_full_pipeline import predict_no_vol
from segmentation_and_depth.src.models.predict_model import predict as predict_seg_depth
from volume_estimation.src.models_2_input_vol.predict_full_pipeline import predict_with_vol

In [6]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

rtde_c, rtde_r, rtde_io_set = connect_robot()

print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(10)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")
gripper.move(45)

Activating Gripper
Gripper activated


True

In [ ]:
pick_up_pos_1 = [-0.20933143110289607,
 0.4007959363962377,
 0.14113332170567885,
 -1.559202807626822,
 0.03734946834843037,
 0.004841554724431051]

pick_up_pos_2 = [-0.23954717580833165,
 0.400795552537626,
 0.07130884774090128,
 -1.5591976930082265,
 0.037387010510812266,
 0.004880320202878557]

pick_up_pos_4 = [-0.23473853377504333,
 0.47152953670488773,
 0.053494416798714306,
 -1.5710858755788153,
 0.017660070482072698,
 0.01776629011841677] # gripping_pos

move_to_pouring_pos_1 = [-0.23473853377504333,
 0.47152953670488773,
 0.183494416798714306,
 -1.5710858755788153,
 0.017660070482072698,
 0.01776629011841677]

move_to_pouring_pos_2 = [-0.22901634283841804,
 0.41998903523904,
 0.2949162372788959,
 -1.5751694238784661,
 -0.21652305912711,
 0.25229985538985844]

move_to_pouring_pos_3 = [-0.3979018097425171,
 0.1980390973153733,
 0.28823304066468763,
 -1.3004855590335682,
 -1.1848976095494719,
 1.213794766836109]

start_pos_horizontal = [-0.36498883237255164,
 0.24715099595577825+0.003,
 0.17519255890890678,
 -1.221741554832158,
 -1.213214812915213,
 1.2457124592502689]

start_pos_bottle_rotated = [-0.36498143802471317,
 0.2501417037919356,
 0.17515693872394192,
 -1.4078543077400933,
 -1.0821978511944732,
 1.4284556202410212]

picture_pos = 

In [ ]:
pos = rtde_r.getActualTCPPose()
pos

In [2]:
def find_best_parameters(input_start_vol, input_target_vol):
    input_target_vol -= 20 # correction factor
    summary_flask = pd.read_csv('../pouring_simulation/output/summary_medium_final.csv', skiprows=[0,1])
    min_loss = 100000

    for j in range(len(summary_flask)):
        loss = abs(summary_flask.iloc[j,5] - input_start_vol) + abs(summary_flask.iloc[j,7] - input_target_vol) #+ summary_flask.iloc[j,8]*0.3
        if loss < min_loss:
            min_loss = loss
            stop_angle = (summary_flask.iloc[j,3])
            stop_time = (summary_flask.iloc[j,4])

    return stop_angle, stop_time

In [ ]:
path = capture_image()
print("Path: ", path)
input_start_vol = predict_with_vol(path, save_segmentation=True, save_depth=True, predict_volume=True)

In [3]:
stop_angle, stop_time = find_best_parameters(99, 22)

print("Minimum loss stop angle: ", stop_angle)
print("Minimum loss stop time: ", stop_time)

Minimum loss stop angle:  72.0
Minimum loss stop time:  0.2
